In [1]:
import pickle

import pandas as pd
import numpy as np
import bokeh.layouts
import bokeh.models
import bokeh.plotting
import bokeh.palettes
import bokeh.plotting
import bokeh.transform
import bokeh.embed
import json

import sp_project.app_state
from sp_project.notebook_tools import default_clients
import sp_project.data_collection.openweather_prediction as weatherprediction
from sp_project.data_preparation.create_model import *
from sp_project.data_preparation.prediction_preparation import *

bokeh.io.output_notebook()

/Users/missd/.pyenv/versions/3.10.4/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Loading BokehJS ...

In [2]:
lon = 7.70
lat = 46.42

In [3]:
with open("../../src/sp_project/backend_server/assets/prediction-model.pickle", "rb") as fh:
    model = pickle.load(fh)

In [4]:
async with default_clients() as (OWclient, db):
    app_state = sp_project.app_state.AppState(ow_client=OWclient, db_client=db, model=model)
    result = await extract_predictions_daily(app_state, lon=lon, lat=lat)
    
    features = prepare_prediction_features(result, lat)
    prediction = energy_prediction(app_state.model, features)

In [5]:
prediction.head()

,wind,solar,nuclear,water_reservoir,water_river,water_pump,total
dt,,,,,,,
2023-05-22 11:00:00+00:00,50.701775,1720.700594,835.213454,2776.526120,1078.894261,1794.408359,8256.444563
2023-05-23 11:00:00+00:00,91.754426,2491.631309,1281.097078,3881.737844,1623.333347,2588.951523,11958.505527
2023-05-24 11:00:00+00:00,60.889794,1562.789874,1728.635453,2792.431732,1048.287811,1829.389287,9022.423952
2023-05-25 11:00:00+00:00,27.057121,749.934193,1759.038594,1750.267152,515.367076,1092.139304,5893.803439
2023-05-26 11:00:00+00:00,18.069101,185.010586,2549.316432,1215.217343,208.191080,734.197616,4910.002158


In [6]:
prediction_source = bokeh.models.ColumnDataSource(prediction)

column_names = "nuclear solar wind water_reservoir water_pump water_river".split()
colors = "#D55E00 #F0E442 #BBBBBB #009E73 #0072B2 #56B4E9".split()

middle = len(prediction) // 2
selection_range = 2

# Selected-Part-Plot
p = bokeh.plotting.figure(
    height=300,
    width=800,
    x_axis_type="datetime",
    x_axis_location="above",
    x_range=(prediction.index[middle - selection_range], prediction.index[middle + selection_range]),
)

p.yaxis.axis_label = 'Energy Production [MW]'

p.varea_stack(
    x='dt',
    stackers=column_names,
    source=prediction_source,
    color=colors,
    legend_label=column_names,
)

# Overview-Plot
overview = bokeh.plotting.figure(
    title="Drag the middle and edges of the selection box to change the range above",
    height=130,
    width=800,
    x_axis_type="datetime",
    y_range=p.y_range,
    y_axis_type=None,
    tools="hover",
    toolbar_location=None,
)

overview.varea_stack(
    x='dt',
    stackers=column_names,
    source=prediction_source,
    color=colors,
)

p.legend.location = "top_left"

range_tool = bokeh.models.RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

overview.ygrid.grid_line_color = None
overview.add_tools(range_tool)

overview_fig = bokeh.layouts.column(p, overview)

bokeh.plotting.show(overview_fig)

In [11]:
x = round(prediction.iloc[:, :-1].sum(), 2)

data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'country'})
data['angle'] = data['value'] / data['value'].sum() * 2 * np.pi
data['color'] = "#BBBBBB #F0E442 #D55E00 #009E73 #0072B2 #56B4E9".split()

p = bokeh.plotting.figure(
    height=350,
    title="Average Production for the next 8 Days",
    toolbar_location=None,
    tools="hover",
    tooltips="@country: @value",
    x_range=(-0.5, 1.0)
)

p.wedge(x=0, y=1, radius=0.4,
        start_angle=bokeh.transform.cumsum('angle', include_zero=True), end_angle=bokeh.transform.cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

pie_fig = bokeh.layouts.column(p)

bokeh.plotting.show(pie_fig)

### Save the Graphics as JSON-Files

In [8]:
with open("energy_prediction_overview.json", "wt", encoding="UTF-8") as fh:
    json.dump(bokeh.embed.json_item(overview_fig), fh)

In [9]:
with open("energy_prediction_pieplot.json", "wt", encoding="UTF-8") as fh:
    json.dump(bokeh.embed.json_item(pie_fig), fh)

## Jupyter-Notebook Footer-Info

In [10]:
import os
import platform
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Darwin | 22.4.0
Datetime: 2023-05-22 09:39:49
Python Version: 3.10.4
-----------------------------------
